In [3]:
# lineitems = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(sys.arg[1])
path="./tpch_tables_scale_0.1/"
# path is where you have the folder. It can be a distributed path like S3, gc or hdfs

customer = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"customer.tbl")
orders = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"orders.tbl")
lineitems = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"lineitem.tbl")
part = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"part.tbl")
supplier = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"supplier.tbl")
partsupp = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"partsupp.tbl")
region = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"region.tbl")
nation = sqlContext.read.format('csv').options(header='true', inferSchema='true',  sep ="|").load(path+"nation.tbl")

In [2]:
customer.show()

+-------+------------------+--------------------+---------+---------------+-------+----------+--------------------+
|CUSTKEY|              NAME|             ADDRESS|NATIONKEY|          PHONE|ACCBATL|MKTSEGMENT|             COMMENT|
+-------+------------------+--------------------+---------+---------------+-------+----------+--------------------+
|      1|Customer#000000001|   IVhzIApeRb ot,c,E|       15|25-989-741-2988| 711.56|  BUILDING|to the even, regu...|
|      2|Customer#000000002|XSTf4,NCwDVaWNe6t...|       13|23-768-687-3665| 121.65|AUTOMOBILE|l accounts. blith...|
|      3|Customer#000000003|        MG9kdTD2WBHm|        1|11-719-748-3364|7498.12|AUTOMOBILE| deposits eat sly...|
|      4|Customer#000000004|         XxVSJsLAGtn|        4|14-128-190-5944|2866.83| MACHINERY| requests. final,...|
|      5|Customer#000000005|KvpyuHCplrB84WgAi...|        3|13-750-942-6364| 794.47| HOUSEHOLD|n accounts will h...|
|      6|Customer#000000006|sKZz0CsnMD7mp4Xd0...|       20|30-114-968-49

In [4]:
orders.show()

+--------+-------+-----------+----------+-------------------+---------------+---------------+-------------+--------------------+
|ORDERKEY|CUSTKEY|ORDERSTATUS|TOTALPRICE|          ORDERDATE| ORDER-PRIORITY|          CLERK|SHIP-PRIORITY|             COMMENT|
+--------+-------+-----------+----------+-------------------+---------------+---------------+-------------+--------------------+
|       1|   3691|          O| 194029.55|1996-01-02 00:00:00|          5-LOW|Clerk#000000951|            0|nstructions sleep...|
|       2|   7801|          O|  60951.63|1996-12-01 00:00:00|       1-URGENT|Clerk#000000880|            0| foxes. pending a...|
|       3|  12332|          F| 247296.05|1993-10-14 00:00:00|          5-LOW|Clerk#000000955|            0|sly final account...|
|       4|  13678|          O|  53829.87|1995-10-11 00:00:00|          5-LOW|Clerk#000000124|            0|sits. slyly regul...|
|       5|   4450|          F| 139660.54|1994-07-30 00:00:00|          5-LOW|Clerk#000000925|    

In [5]:
lineitems.show()

+--------+-------+-------+----------+--------+-------------+--------+----+----------+----------+-------------------+-------------------+-------------------+-----------------+--------+--------------------+
|ORDERKEY|PARTKEY|SUPPKEY|LINENUMBER|QUANTITY|EXTENDEDPRICE|DISCOUNT| TAX|RETURNFLAG|LINESTATUS|           SHIPDATE|         COMMITDATE|        RECEIPTDATE|     SHIPINSTRUCT|SHIPMODE|             COMMENT|
+--------+-------+-------+----------+--------+-------------+--------+----+----------+----------+-------------------+-------------------+-------------------+-----------------+--------+--------------------+
|       1|  15519|    785|         1|      17|     24386.67|    0.04|0.02|         N|         O|1996-03-13 00:00:00|1996-02-12 00:00:00|1996-03-22 00:00:00|DELIVER IN PERSON|   TRUCK|egular courts abo...|
|       1|   6731|    732|         2|      36|     58958.28|    0.09|0.06|         N|         O|1996-04-12 00:00:00|1996-02-28 00:00:00|1996-04-20 00:00:00| TAKE BACK RETURN|    MA

In [9]:
import sys
from operator import add

import requests

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import lit

In [10]:
# Question 1
# Implement a pyspark code that can find out the top-10 sold products. 
lines = lineitems.select("ORDERKEY", "PARTKEY").withColumn("COUNT", lit(1)).groupBy("PARTKEY").agg(func.sum("COUNT"))

lines.show()

+-------+----------+
|PARTKEY|sum(COUNT)|
+-------+----------+
|  14832|        40|
|    471|        26|
|   5518|        25|
|   2866|        27|
|   2122|        31|
|   4900|        31|
|   5803|        31|
|  13832|        25|
|  13289|        41|
|   8592|        31|
|   7754|        30|
|   4101|        36|
|  11458|        34|
|   7833|        41|
|   9465|        33|
|  12940|        28|
|   7554|        27|
|  16339|        25|
|  14450|        30|
|  16503|        35|
+-------+----------+
only showing top 20 rows



In [11]:
lines.orderBy("sum(COUNT)", ascending=False).limit(10).show()

+-------+----------+
|PARTKEY|sum(COUNT)|
+-------+----------+
|  10620|        56|
|   6140|        54|
|  15584|        52|
|   8051|        52|
|  10715|        51|
|  10597|        51|
|   2292|        51|
|  14422|        50|
|  17670|        50|
|  19444|        50|
+-------+----------+



In [16]:
# ---------------------------------------------------------------------------
# Question 2

# Find the top-10 customers based on the number of products ordered.

cus_order = orders.select("CUSTKEY", "ORDERKEY").withColumn(
    'ORDER COUNT', lit(1)).groupBy('CUSTKEY').agg(func.sum('ORDER COUNT'))

result_2 = cus_order.orderBy('sum(ORDER COUNT)', ascending=False).limit(10).show()


+-------+----------------+
|CUSTKEY|sum(ORDER COUNT)|
+-------+----------------+
|   8761|              36|
|  11998|              36|
|   9454|              35|
|    388|              35|
|   3151|              35|
|   4339|              35|
|   8362|              35|
|   4804|              34|
|   3346|              34|
|  10354|              34|
+-------+----------------+



In [23]:
customer_orders = orders.select('ORDERKEY', 'CUSTKEY')

In [24]:
order_items = lineitems.select('ORDERKEY', 'QUANTITY').groupBy('ORDERKEY').agg(func.sum('QUANTITY'))

In [28]:
cus_order_items = customer_orders.join(order_items, customer_orders.ORDERKEY == order_items.ORDERKEY)\
.groupBy('CUSTKEY').agg(func.sum('sum(QUANTITY)'))

In [29]:
cus_order_items.show()

+-------+------------------+
|CUSTKEY|sum(sum(QUANTITY))|
+-------+------------------+
|    463|              1794|
|   2659|              2069|
|  12799|              1943|
|   5156|              1393|
|  11141|               175|
|  13840|              1934|
|   9427|              3052|
|   7982|               398|
|  11317|              1804|
|   9376|              1852|
|  11458|              2766|
|   3749|              1148|
|   4900|              2344|
|   6466|              1308|
|   6658|              1727|
|   1088|               633|
|  12046|              2863|
|   1238|              1373|
|  10817|              1362|
|   3175|              2231|
+-------+------------------+
only showing top 20 rows



In [27]:
cus_order_items.orderBy('sum(sum(QUANTITY))', ascending=False).limit(10).show()

+-------+------------------+
|CUSTKEY|sum(sum(QUANTITY))|
+-------+------------------+
|   8362|              4082|
|   9454|              3870|
|    346|              3817|
|   6958|              3760|
|   1105|              3737|
|  14707|              3710|
|  11998|              3709|
|  14398|              3670|
|   8542|              3660|
|   8761|              3658|
+-------+------------------+



In [36]:
# ---------------------------------------------------------------------------
# Question 3
# Find the top-10 customers that have ordered products from the same supplier. 
partsupp_keys=partsupp.select("PARTKEY", "SUPPKEY")
order_parts = lineitems.select("ORDERKEY", "PARTKEY")
customer_orders = orders.select("ORDERKEY", "CUSTKEY")

In [37]:
customer_parts = customer_orders.join(order_parts, customer_orders.ORDERKEY == order_parts.ORDERKEY , 'full' )\
    .drop('ORDERKEY')

customer_parts.show()


+-------+-------+
|CUSTKEY|PARTKEY|
+-------+-------+
|   5524|   5356|
|   5524|  11184|
|   5524|  16118|
|  14689|  10690|
|  14689|   3685|
|  14689|  18035|
|  14689|  12397|
|  11131|  14993|
|  11131|    462|
|  11131|  10351|
|  11131|  15300|
|  11131|  16580|
|  11131|  11463|
|   4237|  16868|
|   4237|  11944|
|   8206|   4179|
|   8206|   4226|
|   8206|  13455|
|   8206|  11810|
|   8206|    617|
+-------+-------+
only showing top 20 rows



In [38]:
customer_supplier=customer_parts.join(partsupp_keys, customer_parts.PARTKEY == partsupp.PARTKEY , 'full'  )\
                    .drop('PARTKEY')

customer_supplier.withColumn("COUNT", lit(1))\
                .groupBy("CUSTKEY", "SUPPKEY")\
                .agg(func.sum("COUNT")).orderBy("sum(COUNT)", ascending=False).limit(10).show()


+-------+-------+----------+
|CUSTKEY|SUPPKEY|sum(COUNT)|
+-------+-------+----------+
|   4567|    844|         7|
|   4792|    592|         6|
|  11809|     17|         6|
|  14767|      8|         6|
|   2173|    572|         6|
|   6139|    233|         6|
|    874|    430|         6|
|    154|    380|         5|
|   6889|    729|         5|
|   8794|    545|         5|
+-------+-------+----------+



In [ ]:
# ---------------------------------------------------------------------------
# Question 4 and 5
# Find the customers who have not ordered products from their own country and have ordered only foreign products. 


